In [ ]:
!pip install transformers
!pip install requests
!pip install beautifulsoup4
!pip install nltk
!pip install sentencepiece
!pip install torch
!pip install tqdm

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 94.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 27.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 84.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 56.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 35.1 MB/s eta 0:00:00


In [ ]:
from transformers import pipeline
import requests
from bs4 import BeautifulSoup
import nltk
from nltk import tokenize
import re
import json
import torch
from tqdm import tqdm

In [ ]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
config = {
    "to_french": False,
    "url_number": 200,
    "translator_model_en_to_fr": "Helsinki-NLP/opus-mt-en-fr",
    "text2instruction_model": "pszemraj/bart-base-instructiongen-LongForm",
    "summarizer_model": "pszemraj/led-base-book-summary",
}

In [ ]:
translator = pipeline("translation_en_to_fr", model=config["translator_model_en_to_fr"])
pipe_bart_base = pipeline("text2text-generation", model=config["text2instruction_model"])
summarizer = pipeline("summarization", config["summarizer_model"], device=0 if torch.cuda.is_available() else -1,)

/usr/local/lib/python3.10/dist-packages/transformers/models/marian/tokenization_marian.py:194: UserWarning: Recommended: pip install sacremoses.
  warnings.warn("Recommended: pip install sacremoses.")


'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: d024ffc6-eb1d-4787-82e2-edc48dc6c93c)')' thrown while requesting HEAD https://huggingface.co/pszemraj/bart-base-instructiongen-LongForm/resolve/main/merges.txt


In [ ]:
def translate_to_french(text):
    translation = translator(text, max_length=len(text))
    return translation[0]['translation_text']

def inference(text):
    params = {
        "num_beams": 2,
        "early_stopping": True,
        "max_length": 100,
        "truncation": True,
    }
    output_bart_base = pipe_bart_base(text, **params)[0]["generated_text"]
    return output_bart_base

def clean_text(text):
    cleaned_text = re.sub(r"[^a-zA-Z0-9\s]", "", text)
    return cleaned_text.strip()

def get_wikipedia_page_text(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    content_text = soup.select("div.mw-parser-output > p")
    text = '\n'.join(para.text for para in content_text)
    result = summarizer(
           clean_text(text),
           min_length=8,
           max_length=256,
           no_repeat_ngram_size=3,
           encoder_no_repeat_ngram_size=3,
           repetition_penalty=3.5,
           num_beams=4,
           do_sample=False,
           early_stopping=True,
    )
    return result[0]['summary_text']

def chunk_text(text, chunk_size):
    sentences = tokenize.sent_tokenize(text)
    return [sentences[i:i + chunk_size] for i in range(0, len(sentences), chunk_size)]

def generate_json(urls):
  datas = []
  texts = []
  for url in tqdm(urls, desc="Processing URLs"):
    text = get_wikipedia_page_text(url)
    chunks = chunk_text(text, 3)
    texts.extend([" ".join(chunk) for chunk in chunks])
  for text in tqdm(texts, desc="Generating datas"):
    instruction = inference(text)
    output = text
    if config["to_french"] == True:
      instruction = translate_to_french(instruction)
      output = translate_to_french(output)
    datas.append(
        {
            "instruction": instruction,
            "output": output
        }
    )
  return datas

def generate_dataset():
  urls = []
  for _ in tqdm(range(config["url_number"]), desc="Requesting URLs"):
      response = requests.get("https://en.wikipedia.org/wiki/Special:Random")
      urls.append(response.url)
  dataset = generate_json(urls)
  with open("output.json", "w") as f:
    json.dump(dataset, f)
  with open('output.txt', 'w') as txt_file:
      for entry in dataset:
          txt_file.write(entry["output"])
          txt_file.write('\n')

In [ ]:
generate_dataset()

Processing URLs:   5%|▌         | 10/200 [00:12<03:26,  1.09s/it]/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
Your max_length is set to 256, but your input_length is only 125. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)
Generating datas: 100%|██████████| 279/279 [06:04<00:00,  1.31s/it]


NameError: ignored